In [1]:
import numpy as np
import pandas as pd

from PIL import Image
from transformers import CLIPProcessor, CLIPModel

In [2]:
artemis_data = pd.read_csv("//content/drive/MyDrive/CS 2756 Project/artemis_sample.csv")

In [3]:
artemis_data

,art_style,painting,emotion,utterance,repetition
0,Mannerism_Late_Renaissance,agnolo-bronzino_lucrezia-di-cosimo,awe,Nice painting with nice use of color.,5
1,Realism,aleksey-savrasov_autumn-forest-kuntsevo-damn-p...,something else,the leafless tree makes me feel naked.,5
2,Symbolism,nicholas-roerich_himalayas-28,fear,These mountains with their stark and sharp lin...,6
3,Pointillism,paul-signac_port-of-la-rochelle-1921,contentment,I feel contentment because the painting looks ...,5
4,Abstract_Expressionism,richard-tuttle_20-pearls-5-2003,contentment,The shape of the object looks interesting. It...,10
5,Realism,vladimir-makovsky_a-girl-with-geese-1875,contentment,The woman's red color seems to go so well to e...,5
6,Post_Impressionism,moise-kisling_young-girl-with-long-hair-1942,sadness,Girl thinks that 11-year old boys are disgusting.,5
7,Baroque,mestre-ataã­de_nossa-senhora-do-carmo-e-s-o-si...,contentment,The child blessing the man and the beautiful l...,5
8,Art_Nouveau_Modern,aubrey-beardsley_madame-rejane-1894,contentment,The colors and lines are soft and easy to look...,5
9,Symbolism,nicholas-roerich_himalayas-59,fear,I see the devil standing a top bald mountain! ...,6


In [ ]:
!unzip /content/drive/MyDrive/'CS 2756 Project'/artemis_sample.zip

In [27]:
emotions = artemis_data.emotion.unique()

In [36]:
emotions

array(['awe', 'something else', 'fear', 'contentment', 'sadness',
       'excitement', 'disgust', 'amusement'], dtype=object)

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import clip
import torch

clip.available_models()

In [130]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/16', device)


In [131]:
# Calculate features

for i, sample in artemis_data.iterrows():

    try:
        image = Image.open(f"artemis_sample/{sample[0]}/{sample[1]}.jpg")
    except FileNotFoundError:
        continue

    image_input = preprocess(image).unsqueeze(0).to(device)
    text_inputs = torch.cat([clip.tokenize(f"{sample[3]}. This image make a person feel {emotion}.") for emotion in emotions]).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)

    # Pick the top 5 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity[0].topk(5)

    print(f"Actual Emotion: {sample[2]}")
    print("Top predictions:")
    for value, index in zip(values, indices):
        print(f"{emotions[index]:>16s}: {100 * value.item():.2f}%")
    print()


Actual Emotion: awe
Top predictions:
         disgust: 34.20%
  something else: 12.95%
       amusement: 11.32%
            fear: 11.23%
     contentment: 10.15%

Actual Emotion: something else
Top predictions:
  something else: 18.22%
            fear: 14.61%
     contentment: 13.97%
             awe: 12.63%
         sadness: 12.33%

Actual Emotion: fear
Top predictions:
     contentment: 23.21%
  something else: 15.19%
            fear: 11.19%
       amusement: 11.19%
         sadness: 10.75%

Actual Emotion: contentment
Top predictions:
  something else: 15.76%
       amusement: 15.39%
            fear: 14.25%
      excitement: 13.91%
             awe: 12.42%

Actual Emotion: contentment
Top predictions:
         disgust: 29.55%
      excitement: 10.98%
         sadness: 10.90%
       amusement: 10.83%
     contentment: 10.82%

Actual Emotion: contentment
Top predictions:
             awe: 21.70%
     contentment: 19.38%
  something else: 15.77%
      excitement: 11.28%
       amuse